# CIF Parser + DuckDB Integration

This notebook demonstrates how the loop iterator enables easy integration with DuckDB for powerful SQL-based analysis of CIF data.

**Requirements:**
```bash
uv sync --extra examples
# or: pip install pandas duckdb
```

In [ ]:
import cif_parser

## Sample CIF Data

A crystal structure with atomic coordinates in a loop.

In [ ]:
cif_content = """
data_crystal_structure
_cell_length_a    10.000
_cell_length_b    10.000
_cell_length_c    10.000

loop_
_atom_site_label
_atom_site_type_symbol
_atom_site_fract_x
_atom_site_fract_y
_atom_site_fract_z
_atom_site_occupancy
C1   C   0.1234  0.2345  0.3456  1.00
C2   C   0.2345  0.3456  0.4567  1.00
N1   N   0.3456  0.4567  0.5678  0.95
O1   O   0.4567  0.5678  0.6789  1.00
O2   O   0.5678  0.6789  0.7890  0.90
H1   H   0.6789  0.7890  0.8901  1.00
H2   H   0.7890  0.8901  0.9012  1.00
"""

## Parse the CIF

In [ ]:
doc = cif_parser.parse(cif_content)
block = doc.first_block()
assert block is not None, "Expected block in CIF document"
loop = block.get_loop(0)
assert loop is not None, "Expected loop in block"

print(f"Found loop with {len(list(loop))} rows")

## Method 1: Simple Python Filtering

No external dependencies needed - just iterate over the loop.

In [ ]:
# Re-get loop since we consumed it above
loop = block.get_loop(0)

carbon_atoms = [row for row in loop if row["_atom_site_type_symbol"].text == "C"]
print(f"Found {len(carbon_atoms)} carbon atoms:")
for atom in carbon_atoms:
    label = atom["_atom_site_label"].text
    x = atom["_atom_site_fract_x"].numeric
    print(f"  {label}: x={x}")

## Method 2: Using pandas DataFrame

Convert the loop to a pandas DataFrame for more complex analysis.

In [ ]:
import pandas as pd

# Re-get loop
loop = block.get_loop(0)

# Convert loop to list of dicts, extracting values
rows = []
for row in loop:
    row_data = {}
    for tag, value in row.items():
        if value.is_numeric:
            row_data[tag] = value.numeric
        elif value.is_text:
            row_data[tag] = value.text
        else:
            row_data[tag] = None
    rows.append(row_data)

df = pd.DataFrame(rows)
df

In [ ]:
print("Summary statistics for occupancy:")
df["_atom_site_occupancy"].describe()

## Method 3: Using DuckDB for SQL Queries

DuckDB allows powerful SQL queries directly on Python data structures.

In [ ]:
import duckdb

# Re-get loop and convert
loop = block.get_loop(0)
rows = []
for row in loop:
    row_data = {}
    for tag, value in row.items():
        if value.is_numeric:
            row_data[tag] = value.numeric
        elif value.is_text:
            row_data[tag] = value.text
        else:
            row_data[tag] = None
    rows.append(row_data)

print("Data ready for DuckDB queries")

In [ ]:
# Group by atom type with statistics
result = duckdb.query("""
    SELECT
        _atom_site_type_symbol as type,
        COUNT(*) as count,
        AVG(_atom_site_occupancy) as avg_occupancy,
        MIN(_atom_site_fract_z) as min_z,
        MAX(_atom_site_fract_z) as max_z
    FROM rows
    GROUP BY _atom_site_type_symbol
    ORDER BY count DESC
""").to_df()
result

In [ ]:
# Find atoms with fractional z > 0.5
high_z = duckdb.query("""
    SELECT
        _atom_site_label,
        _atom_site_type_symbol,
        _atom_site_fract_z
    FROM rows
    WHERE _atom_site_fract_z > 0.5
    ORDER BY _atom_site_fract_z
""").to_df()
high_z

In [ ]:
# Atoms with partial occupancy (< 1.0)
partial = duckdb.query("""
    SELECT *
    FROM rows
    WHERE _atom_site_occupancy < 1.0
""").to_df()
partial

## Summary

The loop iterator enables:

1. **Simple Python iteration:** `for row in loop`
2. **Easy conversion to pandas DataFrame**
3. **SQL queries via DuckDB on CIF data**
4. **Powerful data analysis workflows**

All with just: `list(loop)`